### Device Activation Workflow

This notebook will be used to outline the steps required in order to create a workflow to add a new device, test connectivity and activate that device. 

First, two libraries of the MSA Python SDK must be imported: Device and Variables. 

In [1]:
from msa_sdk.device import Device
from msa_sdk.variables import Variables

The Variables library controls how variables are passed between the tasks and the orchestration engine. While the Device library is used to perform any MSA function to manage a device. 

In the first task, a device will be created. This is done by creating a _Device_ object and calling its _create()_ method. In order to create a new device object, a number of variables must be declared initially.

This is done by creating a new _Variables_ object and then using its _add()_ method to create the new variables. Once all the variables are created, they are submitted to the orchestration engine using the _task_call()_ method

In [2]:
dev_var = Variables()

dev_var.add('customer_id', var_type='Integer')
dev_var.add('managed_device_name')
dev_var.add('manufacturer_id', var_type='Integer')
dev_var.add('model_id', var_type='Integer')
dev_var.add('device_ip_address', var_type='IP Address')
dev_var.add('login')
dev_var.add('password', var_type='Password')
dev_var.add('password_admin', var_type='Password')

context = Variables.task_call(dev_var)

Next, the new device object will be created by calling the _Device_ constructor. The device can then be created in the MSA by calling its _create()_ method.

For future tasks, the _device_id_ will be stored in context and the task will end with a 'Task OK' message. All the variables created in this task will be made available to the next task.

In [ ]:
new_device = Device(context['customer_id'], context['managed_device_name'], context['manufacturer_id'],context['model_id'], context['login'], context['password'], context['password_admin'],context['device_ip_address'])
new_device.create()
context['device_id'] = new_device.device_id

print(new_device.process_content('ENDED', 'Task OK', context, True))

The second task will test network connectivity by pinging the device. This is done by calling the _Device_ object's _ping_ method.

In [ ]:
new_device = Device(device_id=context['device_id'])
new_device.read()
new_device.ping(new_device.management_address)

print(new_device.process_content('ENDED', 'Pinging IP: ' + new_device.management_address + ' successfully', context, True))

Finally, the device will be activated on the MSA by calling the _Device_ object's _initial_provisioning()_ method.

In [ ]:
new_device = Device(device_id=context['device_id'])
new_device.initial_provisioning()

print(new_device.process_content('ENDED', 'Device: ' + str(new_device.device_id) + ' provisioned successfully', context, True))